## Task 1. Bron–Kerbosch algorithm

In [1]:
def bron_kerbosch(R, P, X, graph, cliques):
    if not P and not X:
        cliques.append(R)
    for v in P.copy():
        bron_kerbosch(
            R.union([v]), 
            P.intersection(graph[v]), 
            X.intersection(graph[v]), 
            graph, 
            cliques,
        )
        P.remove(v)
        X.add(v)

def find_all_cliques(graph):
    P = set(graph.keys())  # содержит все вершины графа
    R = set() # Содержит текущую клику
    X = set() # Содержит вершины, которые уже рассмотрены
    cliques = [] # Список максимальных клик
    bron_kerbosch(R, P, X, graph, cliques) 
    return cliques


In [2]:
graph = {
    1: [2, 5],
    2: [1, 3, 5],
    3: [2, 4],
    4: [3, 5, 6],
    5: [1, 2, 4],
    6: [4],
}

cliques = find_all_cliques(graph)
print(cliques)


[{1, 2, 5}, {2, 3}, {3, 4}, {4, 5}, {4, 6}]


## Task 2. Random Forest

In [3]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth # максимальная глубина дерева
        self.min_samples_split = min_samples_split # минимальное количество объектов для разделения в узле
        self.split_feature = None
        self.split_value = None
        self.left_child = None
        self.right_child = None
        self.label = None

    def fit(self, X, y):
        # Если максимальная глубина не была указана, она устанавливается равной числу признаков в данных X.
        if self.max_depth is None:
            self.max_depth = X.shape[1]

        # Если число образцов в выборке меньше минимального числа для разделения, 
        # либо достигнута максимальная глубина дерева, либо все метки y одинаковы, 
        # то задается метка для этого узла и завершается метод.
        if X.shape[0] < self.min_samples_split or self.max_depth == 0 or len(np.unique(y)) == 1:
            self.label = np.bincount(y).argmax()
            return

        best_feature = None
        best_value = None
        best_score = np.inf

        for feature in range(X.shape[1]):
            for value in np.unique(X[:, feature]):
                left_y = y[X[:, feature] < value]
                right_y = y[X[:, feature] >= value]
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                score = (len(left_y) * self.gini(left_y) + len(right_y) * self.gini(right_y)) / len(y)
                if score < best_score:
                    best_feature = feature
                    best_value = value
                    best_score = score

        self.split_feature = best_feature
        self.split_value = best_value

        # Создание левого и правого потомков дерева с уменьшенной максимальной глубиной и минимальным 
        # количеством образцов для разделения, и рекурсивный вызов метода fit для каждого из них.
        self.left_child = DecisionTree(max_depth=self.max_depth-1, min_samples_split=self.min_samples_split)
        self.right_child = DecisionTree(max_depth=self.max_depth-1, min_samples_split=self.min_samples_split)
        self.left_child.fit(X[X[:, best_feature] < best_value], y[X[:, best_feature] < best_value])
        self.right_child.fit(X[X[:, best_feature] >= best_value], y[X[:, best_feature] >= best_value])


    def predict(self, X):
        # Если задана метка для узла, то возвращается массив этой метки для всех примеров в X.
        if self.label is not None:
            return np.array([self.label] * X.shape[0])
        else:
            y = np.zeros(X.shape[0])
            y[X[:, self.split_feature] < self.split_value] = self.left_child.predict(X[X[:, self.split_feature] < self.split_value])
            y[X[:, self.split_feature] >= self.split_value] = self.right_child.predict(X[X[:, self.split_feature] >= self.split_value])
            return y

    def gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        p = counts / len(y)
        return 1 - np.sum(p ** 2)


In [4]:
class RandomForest:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators # количество деревьев
        self.max_depth = max_depth # максимальная глубина каждого дерева
        self.min_samples_split = min_samples_split # минимальное количество выборок для разделения 
        self.max_features = max_features # максимальное количество признаков для узлов деревьев
        self.trees = []

    def fit(self, X, y):
        if self.max_features is None:
            self.max_features = int(np.sqrt(X.shape[1]))
        for i in range(self.n_estimators):
            sample_indices = np.random.choice(X.shape[0], X.shape[0])
            feature_indices = np.random.choice(X.shape[1], self.max_features, replace=False)
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X[sample_indices][:, feature_indices], y[sample_indices])
            self.trees.append(tree)

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        for tree in self.trees:
            y_pred += tree.predict(X[:, np.random.choice(X.shape[1], self.max_features, replace=False)]) / self.n_estimators
            #print(y_pred)
        if (y_pred <= 1).all():
            return y_pred.round()
        
        y_pred = np.where(y_pred < 1, y_pred, y_pred.round())
        return y_pred.astype(int)

In [12]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForest(n_estimators=1, max_depth=3, max_features=3)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.36666666666666664


In [6]:
y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [7]:
y_pred

array([1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 1, 1, 2, 1, 1, 2, 0, 2,
       1, 2, 2, 2, 2, 2, 0, 0])

In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

data = load_iris()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTree(max_depth=3, min_samples_split=2)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.63


In [9]:
y_pred

array([1., 0., 2., 1., 2., 0., 1., 2., 1., 1., 2., 0., 0., 0., 0., 1., 2.,
       1., 1., 2., 0., 2., 0., 2., 2., 2., 2., 2., 0., 0.])

In [10]:
y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])